# Ket Programming Tutorial

Documentation: https://quantumket.org

## Importing Ket into the notebook

In [ ]:
from ket import *
!ket --version

## Allocating quantum Bit

- `quant`: list of qubits.

In [ ]:
n = 10
qubits = quant(n)  # Allocation n qubits

In [ ]:
qubits

## Printing the quantum state

- `dump`: snapshot of the quantum state.

In [ ]:
quantum_state = dump(qubits)

print(quantum_state.show())  # Print the quantum state

### Rendering Bloch sphere

In [ ]:
qubit = quant()  # Allocate 1 qubit

quantum_state = dump(qubit)
bloch_sphere = quantum_state.sphere()

bloch_sphere.show()

## Applying quantum gates

[Available quantum gates](https://quantumket.org/ket.html#module-ket.gates.quantum_gate.quantum_gate)

In [ ]:
from math import pi

qubit = quant()

H(qubit)
phase(pi/4, qubit)

dump(qubit).sphere().show()

In [ ]:
a, b = quant(2)

H(a)
cnot(a, b)

print(dump(a+b).show())

### Concatenating quantum gates

In [ ]:
XH = H(X)
print(XH)

In [ ]:
qubit = XH(quant())

dump(qubit).sphere().show()

In [ ]:
T = phase(pi/4)
print(T)

In [ ]:
HTH = H(T(H))
print(HTH)

In [ ]:
qubit = HTH(quant())

dump(qubit).sphere().show()

In [ ]:
bell = cnot(H, I)
print(bell)

In [ ]:
qubits = quant(2)

bell(*qubits)

print(dump(qubits).show())

### Adding control qubits

- `with control`: start a controlled scope.

In [ ]:
a, b = quant(2)

H(a)

with control(a):
    X(b)

print(dump(a+b).show())

- `ctrl`: call with control qubits.

In [ ]:
a, b = quant(2)

H(a)

ctrl(a, X, b)

print(dump(a+b).show())

In [ ]:
def bell(a, b):
    ctrl(H(a), X, b)

In [ ]:
qubits = quant(3)

H(qubits[0])

with control(qubits[0]):
    bell(qubits[1], qubits[2])

# ctrl(qubits[0], bell, *qubits[1:])

print(dump(qubits).show())


### Calling inverse quantum gates

- `with inverse`: start an inverse scope.

In [ ]:
a, b = quant(2)

bell(a, b)

Y(a)

with inverse():
    H(a)
    cnot(a, b)

print(dump(a+b).show())


- `adj`: call the inverse operation.

In [ ]:
a, b = quant(2)

bell(a, b)

Y(a)

adj(bell, a, b)

print(dump(a+b).show())


- `with around`: Apply $UVU^\dagger$.

In [ ]:
a, b = quant(2)

with around(bell, a, b):
    Y(a)

print(dump(a+b).show())


## Measuring qubits

- `measure`: Measure a `quant`.

In [ ]:
a, b = quant(2)

bell(a, b)

m_a = measure(a)
m_b = measure(b)

print(m_a.value, m_b.value) # read the measurement results

In [ ]:
qubits = quant(16)

H(qubits)

random_int = measure(qubits)

print(random_int.value)

## Example: Grover's algorithm

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Grover%27s_algorithm_circuit.svg" alt="drawing" width="600"/>

In [ ]:
def diffusion(qubits):
    with around([H, X], qubits):
        ctrl(qubits[1:], Z, qubits[0])

In [ ]:
from math import sqrt

def grover(n, oracle):
    qubits = H(quant(n))

    N = 2**n
    steps = (pi/4)*sqrt(N)

    for _ in range(int(steps)):
        oracle(qubits)
        diffusion(qubits)

    return measure(qubits)  

- `oracle_aux` and `oracle_input_aux`: 
  - $U_f\left|x\right>\left|y\right> = \left|x\right>\left|y\oplus f(x)\right>$
  - $f(x) = \begin{cases} 1, & \text{if } x=w \\ 0, & \text{otherwise} \end{cases}$
  - qubit `aux =` $\left|+\right>$
- `oracle_without_aux`:
  - $U_w\left|x\right> = \begin{cases} -\left|x\right>, & \text{if } x=w \\ \left|x\right>, & \text{otherwise} \end{cases}$

In [ ]:
def oracle_aux(w, qubits):
    with quant() as aux:
        with around([X, H], aux):
            with control(qubits, on_state=w):
                X(aux)
        aux.free()


def oracle_input_aux(w, aux, qubits):
    with control(qubits, on_state=w):
        X(aux)


def oracle_without_aux(w, qubits):  # phase_on(w, qubits) for ket >= 0.4
    n = len(qubits)
    to_flip = qubits.at([i for i, b in enumerate(f'{w:0{n}b}') if b == '0'])

    with around(X, to_flip):
        ctrl(qubits[1:], Z, qubits[0])

In [ ]:
from functools import partial

n = 8
w = 11

result = grover(n, partial(oracle_aux, w))
print(result.value)
print(quantum_code_last()[0]['gate_count'])

In [ ]:
aux = H(X(quant()))
result = grover(n, partial(oracle_input_aux, w, aux))
print(result.value)
print(quantum_code_last()[0]['gate_count'])

In [ ]:
result = grover(n, partial(oracle_without_aux, w))
print(result.value)
print(quantum_code_last()[0]['gate_count'])

In [ ]:
def grover_dump(n, oracle):
    qubits = H(quant(n))

    N = 2**n
    steps = (pi/4)*sqrt(N)

    d = [dump(qubits)]

    for _ in range(int(steps)):
        oracle(qubits)
        diffusion(qubits)
        d.append(dump(qubits))

    return d

In [ ]:
n = 8
w = (2**n)//2
states = grover_dump(n, partial(oracle_without_aux, w))

In [ ]:
import plotly.express as px

px.bar(
    {
        'State': [s for d in states for s, _ in sorted(d.get_quantum_state().items())],
        'Probability (%)': [abs(a)**2*100 for d in states for _, a in sorted(d.get_quantum_state().items())],
        'Step': [i for i, d in enumerate(states) for _ in range(len(d.get_quantum_state()))]
    },
    x='State',
    y='Probability (%)',
    animation_frame='Step'
)

In [ ]:
px.bar(
    {
        'State': ['W', 'Others']*len(states),
        'Probability (%)': [p*100 for d in states for p in [abs(d.get_quantum_state()[w])**2, 1-abs(d.get_quantum_state()[w])**2]],
        'Step': [i for i in range(len(states)) for _ in range(2)]
    },
    x='State',
    y='Probability (%)',
    animation_frame='Step'
)

## Ket runtime

<img src="https://quantumket.org/_images/runtime.png" alt="drawing" width="600"/>


In [ ]:
qubits = H(quant(10))

m = measure(qubits)

print(m)
print(m.value)

In [ ]:
a, b = quant(2)

bell(a, b)

print('Measurement of a =', measure(a).value)
print('Measurement of b =', measure(b).value) # this will raise an error

In [ ]:
from pprint import pprint
from ket import base as libket

a, b = quant(2)
print(f'{(a, b)=}')
print(f'{libket.process_top().pid=}')
print()

bell(a, b)

print('Measurement of a =', measure(a).value)
pprint(quantum_code_last())
print()

print(f'{libket.process_top().pid=}')
pprint(quantum_code())
print('Measurement of b =', measure(b).value)  # this will raise an error


In [ ]:
a, b = quant(2)

bell(a, b)

m_a = measure(a)
m_b = measure(b)

print('Measurement of a =', m_a.value)
print('Measurement of b =', m_b.value)
pprint(quantum_code_last())

## Example: Quantum teleportation

<img src="https://upload.wikimedia.org/wikipedia/commons/d/dc/Quantum_teleportation_circuit.svg" alt="drawing" width="600"/>

[teleport.py](teleport.py)

## Example: Shor's algorithm

In [ ]:
from math import log2, gcd
from random import randint
from ket.plugins import pown
from ket.lib import qft
from functools import reduce


def shor(N):
    # if N is even
    if N % 2 == 0:
        return 2, 'even'

    # if N = a**b
    n = N.bit_length()
    y = int(log2(N))
    for b in range(2, n+1):
        x = y/b
        u1 = int(2**x)
        u2 = u1+1

        if u1**b == N:
            return u1, 'a**b'
        elif u2**b == N:
            return u2, 'a**b'

    for _ in range(n):
        try:
            # guess a factor
            x = randint(2, N-1)
            gcd_x_N = gcd(x, N)
            if gcd_x_N > 1:
                return gcd_x_N, 'luck'

            def quantum_subroutine():
                reg1 = H(quant(n))
                reg2 = pown(x, reg1, N)
                measure(reg2)
                adj(qft, reg1)
                return measure(reg1).value

            r = reduce(gcd, [quantum_subroutine() for _ in range(n)])
            r = 2**n//r

            # classical processing
            if r % 2 == 0 and pow(x, r//2) != -1 % N:
                p = gcd(x**(r//2)-1, N)
                if p != 1 and p != N and p*N//p == N:
                    return p, 'quantum'

                q = gcd(x**(r//2)+1, N)
                if q != 1 and q != N and q*N//q == N:
                    return q, 'quantum'
        except:
            continue

    return N, 'prime or fail'

In [ ]:
for N in range(4, 100):
    factor, message = shor(N)
    print(f'N: {N:2} = {factor:2} * {N//factor:2} ({message})')

# Ket Bitwise Simulator (KBW)

Quantum computer simulator:
- Sparse simulator
- Dense simulator

In [ ]:
from ket import kbw

kbw.use_dense() # Use dense simulator
kbw.use_sparse() # Use sparse simulator (Default)

In [ ]:
from ket import lib

kbw.use_sparse()

n = 64
qubits = lib.w(n)

print(f'{measure(qubits).value:0{n}b}')

print(quantum_exec_time())

In [ ]:
kbw.use_dense()

n = 22
qubits = lib.w(n)

print(f'{measure(qubits).value:0{n}b}')

print(quantum_exec_time())

In [ ]:
kbw.use_sparse()

n = 20
qubits = H(quant(n))

print(f'{measure(qubits).value:0{n}b}')

print(quantum_exec_time())

In [ ]:
kbw.use_dense()

n = 20
qubits = H(quant(n))

print(f'{measure(qubits).value:0{n}b}')

print(quantum_exec_time())

In [ ]:
def benchmark(func, begin, end, timeout):
    result = {}

    for size in range(begin, end):
        try:
            quantum_exec_timeout(timeout)
            func(size)
        except Exception as e:
            print(e)
            break

        result[size] = quantum_exec_time()
        size += 1

    return result


def run_benchmark(func, begin, end, timeout):
    kbw.use_dense()
    dense = benchmark(func, begin, end, timeout)

    kbw.use_sparse()
    sparse = benchmark(func, begin, end, timeout)

    return dense, sparse


def plot_benchmark(func, begin, end, timeout):
    dense, sparse = run_benchmark(func, begin, end, timeout)
    return px.line(
        {
            'Nº Qubits': list(sorted(dense.keys()))+list(sorted(sparse.keys())),
            'Time (s)': [dense[n] for n in sorted(dense.keys())]+[sparse[n] for n in sorted(sparse.keys())],
            'Simulator': ['Dense']*len(dense)+['sparse']*len(sparse),
        },
        x='Nº Qubits',
        y='Time (s)',
        color='Simulator'
    )

In [ ]:
def w_benchmark(size):
    measure(lib.w(size))
    exec_quantum()

plot_benchmark(w_benchmark, begin=3, end=30, timeout=1)

In [ ]:
def h_benchmark(size):
    measure(H(quant(size)))
    exec_quantum()

plot_benchmark(h_benchmark, begin=3, end=30, timeout=1)

In [ ]:
def grover_benchmark(size):
    grover(size, partial(oracle_without_aux, 7))
    exec_quantum()

plot_benchmark(grover_benchmark, begin=3, end=30, timeout=2)

--------

![](https://quantumket.org/_static/ket.svg)

# [quantumket.org](https://quantumket.org)